# 03 Geocoding

In this notebook we geocode our address data.

In [1]:
import pandas as pd
import requests as req
import numpy as np
from shapely import wkt
import time
import geopandas as gpd
import re

### Load and Adjust Data

In [7]:
def load_data_to_geocode(folder="../data/cleaned/"):
    df_fairfax = pd.read_csv(folder+"df_fairfax_cleaned.csv" ,index_col=[0])
    df_connecticut = pd.read_csv(folder+"df_connecticut_cleaned.csv", index_col=[0])
    df_fairfax_addr = df_fairfax[["city", "addr", "county"]].drop_duplicates()
    df_connecticut_addr = df_connecticut[["city", "addr", "county"]].drop_duplicates()
    df_fairfax_addr["state"] = "Virginia"
    df_connecticut_addr["state"] = "Connecticut"
    df_combined = pd.concat([df_fairfax_addr, df_connecticut_addr])
    df_combined["addr"] = df_combined.addr.str.title()
    df_combined["city"] = df_combined.city.str.title()
    return df_combined

#### Utility Functions

In [8]:
def split_digits(street):
    first_split = street.split(' ')[0]
    if re.search('\d+', first_split):
        return ' '.join(street.split(' ')[1:])
    else:
        return street

In [9]:
def match(housenumber):
    match = re.match(r'^\d+', housenumber)
    if match:
        return match.group()
    else: 
        return match

In [10]:
def remove_end_digits(street):
    if "Route" in street:
        return street
    else:
        return re.sub(r'(?:\d+[+-]\d*|[-+]\d+|\d+[A-Za-z]*)(\s|$)', '', street)


In [11]:
def remove_start_digits(street):
    if "Route" in street:
        return street
    else:
        return re.sub(r'(\s|^)(?:\d+[+-]\d*|[-+]\d+|\d+[A-Za-z]*)', '', street)

In [12]:
def put_the_to_front(street):
    if re.search(r"\bThe\b", street):
        street = re.sub(r"\bThe\b", "", street)
        street = street.replace("()", "").strip()
        street = street.strip(",")
        return "The " + street.strip()  
    else:
        return street

In [14]:
def ad_split(street):
    parts = re.split(r"/|&|\\", street)
    if len(parts) > 1:
        first = (parts[0]
                .strip()
                .strip("-")
                .strip(".")
                )
        
        if first.isdigit() or (first == ""):
            return parts[1]
        return parts[0]
    return street

In [15]:
def remove_letters(street):
    if re.search(r"\bB Lane", street):
        return re.sub(r"\b(?!')[a-zA-Z]$", "", street.strip())
    elif re.search(r"Avenue B(\s|$)", street) or re.search(r"Aaron B(\s|$)", street) or re.search(r"Ave B(\s|$)", street):
        return re.sub(r"^[a-zA-Z]\b(?!')", " ", street.strip())
    else:
        return re.sub(r"(^)\b(?!')[a-zA-Z]\b(?!')|\b(?!')[a-zA-Z]\b(?!')($)", '', street)

In [16]:
def remove_after_paran(street):
    return re.sub(r'\(.*?$', "", street)

In [17]:
def abbrv(df_addr):
    return (df_addr.apply(lambda x: re.sub(r"St Joseph", "Saint Joseph", x))
                      .apply(lambda x: re.sub(r"St($|\b)", "Street ", x))
                      .apply(lambda x: re.sub(r'\bUnit(\b|$)\.*', '', x))
                      .apply(lambda x: re.sub(r"Ct($|\b)", "Court ", x))
                      .apply(lambda x: re.sub(r"Crt($|\b)", "Court ", x))
                      .apply(lambda x: re.sub(r"Cor($|\b)", "Corner ", x))
                      .apply(lambda x: re.sub(r"Ctr($|\b)", "Center ", x))
                      .apply(lambda x: re.sub(r"Av($|\s|\b)", "Avenue ", x))
                      .apply(lambda x: re.sub(r"Ave($|\s|\b)", "Avenue ", x))
                      .apply(lambda x: re.sub(r"Apts($|\b)", "Apartments ", x))
                      .apply(lambda x: re.sub(r"Al($|\b)", "Alley ", x))
                      .apply(lambda x: re.sub(r"Tr($|\b)", "Terrace ", x))
                      .apply(lambda x: re.sub(r"Terr($|\b)", "Terrace ", x))
                      .apply(lambda x: re.sub(r"Te($|\b)", "Terrace ", x))
                      .apply(lambda x: re.sub(r"Trce($|\b)", "Trace", x))
                      .apply(lambda x: re.sub(r"Thse($|\b)", "Townhouse ", x))
                      .apply(lambda x: re.sub(r"Pl($|\b)", "Place ", x))
                      .apply(lambda x: re.sub(r"Wy($|\b)", "Way ", x))
                      .apply(lambda x: re.sub(r"Wa($|\b)", "Way ", x))
                      .apply(lambda x: re.sub(r"Ter($|\b)", "Terrace ", x))
                      .apply(lambda x: re.sub(r"Blvd($|\b)", "Boulevard ", x))
                      .apply(lambda x: re.sub(r"Bl($|\b)", "Boulevard ", x))
                      .apply(lambda x: re.sub(r"Bv($|\b)", "Boulevard ", x))
                      .apply(lambda x: re.sub(r"Blv($|\b)", "Boulevard ", x))
                      .apply(lambda x: re.sub(r"Bch($|\b)", "Beach ", x))
                      .apply(lambda x: re.sub(r"Cmn($|\b)", "Common ", x))
                      .apply(lambda x: re.sub(r"Cmns($|\b)", "Commons ", x))
                      .apply(lambda x: re.sub(r"Hol($|\b)", "Hollow ", x))
                      .apply(lambda x: re.sub(r"Hlw($|\b)", "Hollow ", x))
                      .apply(lambda x: re.sub(r"Sq($|\b)", "Square ", x))
                      .apply(lambda x: re.sub(r"Trl($|\b)", "Trail ", x))
                      .apply(lambda x: re.sub(r"Tl($|\b)", "Trail ", x))
                      .apply(lambda x: re.sub(r"Hwy($|\b)", "Highway ", x))
                      .apply(lambda x: re.sub(r"Hi($|\b)", "Highway ", x))
                      .apply(lambda x: re.sub(r"Hgwy($|\b)", "Highway ", x))
                      .apply(lambda x: re.sub(r"Lndg($|\b)", "Landing ", x))
                      .apply(lambda x: re.sub(r"Pt($|\b)", "Point ", x))
                      .apply(lambda x: re.sub(r"Po($|\b)", "Point ", x))
                      .apply(lambda x: re.sub(r"Pro($|\b)", "Professional ", x))
                      .apply(lambda x: re.sub(r"Brk($|\b)", "Brook ", x))
                      .apply(lambda x: re.sub(r"Rdg($|\b)", "Ridge ", x))
                      .apply(lambda x: re.sub(r"So($|\b)", "South ", x))
                      .apply(lambda x: re.sub(r"Tp($|\b)", "Turnpike ", x))
                      .apply(lambda x: re.sub(r"Tpk($|\b)", "Turnpike ", x))
                      .apply(lambda x: re.sub(r"Tpke($|\b)", "Turnpike ", x))
                      .apply(lambda x: re.sub(r"Tnpk($|\b)", "Turnpike ", x))
                      .apply(lambda x: re.sub(r"Bp($|\b)", "Bridgeport ", x))
                      .apply(lambda x: re.sub(r"Br($|\b)", "Bridge ", x))
                      .apply(lambda x: re.sub(r"Pkwy($|\b)", "Parkway ", x))
                      .apply(lambda x: re.sub(r"Pky($|\b)", "Parkway ", x))
                      .apply(lambda x: re.sub(r"Vw($|\b)", "View ", x))
                      .apply(lambda x: re.sub(r"Cv($|\b)", "Cove ", x))
                      .apply(lambda x: re.sub(r"Ave($|\b)", "Avenue ", x))
                      .apply(lambda x: re.sub(r"Dr($|\b)", "Drive ", x))
                      .apply(lambda x: re.sub(r"Cr($|\b)", "Circle ", x))
                      .apply(lambda x: re.sub(r"Ci($|\b)", "Circle ", x))
                      .apply(lambda x: re.sub(r"Cir($|\b)", "Circle ", x))
                      .apply(lambda x: re.sub(r"Cl($|\b)", "Close ", x))
                      .apply(lambda x: re.sub(r"Cswy($|\b)", "Causeway ", x))
                      .apply(lambda x: re.sub(r"Cres($|\b)", "Crescent ", x))
                      .apply(lambda x: re.sub(r"Se($|\b)", "Southeast ", x))
                      .apply(lambda x: re.sub(r"Sw($|\b)", "Southwest ", x))
                      .apply(lambda x: re.sub(r"Nw($|\b)", "Northwest ", x))
                      .apply(lambda x: re.sub(r"Ne($|\b)", "Northeast ", x))
                      .apply(lambda x: re.sub(r"No($|\b)", "North ", x))
                      .apply(lambda x: re.sub(r"Plz($|\b)", "Plaza ", x))
                      .apply(lambda x: re.sub(r"Ptwy($|\b)", "Pathway ", x))
                      .apply(lambda x: re.sub(r"Pth($|\b)", "Path ", x))
                      .apply(lambda x: re.sub(r"Grv($|\b)", "Grove ", x))
                      .apply(lambda x: re.sub(r"Gr($|\b)", "Grove ", x))
                      .apply(lambda x: re.sub(r"Drs($|\b)", "Drive ", x))
                      .apply(lambda x: re.sub(r"Hgts($|\b)", "Heights ", x))
                      .apply(lambda x: re.sub(r"Hghts($|\b)", "Heights ", x))
                      .apply(lambda x: re.sub(r"Hts($|\b)", "Heights ", x))
                      .apply(lambda x: re.sub(r"Ht($|\b)", "Heights ", x))
                      .apply(lambda x: re.sub(r"Hl($|\b)", "Hill ", x))
                      .apply(lambda x: re.sub(r"Hls($|\b)", "Hills ", x))
                      .apply(lambda x: re.sub(r"Pk($|\b)", "Park ", x))
                      .apply(lambda x: re.sub(r"Rdg($|\b)", "Ridge ", x))
                      .apply(lambda x: re.sub(r"Ex($|\b)", "Extension ", x))
                      .apply(lambda x: re.sub(r"Ext($|\b)", "Extension ", x))
                      .apply(lambda x: re.sub(r"Rte($|\b)", "Route ", x))
                      .apply(lambda x: re.sub(r"Rd($|\b)", "Road ", x))
                      .apply(lambda x: re.sub(r"Rvr($|\b)", "River ", x))
                      .apply(lambda x: re.sub(r"La($|\b)", "Lane ", x))
                      .apply(lambda x: re.sub(r"Ln($|\b)", "Lane ", x))
                      .apply(lambda x: re.sub(r"Ldg($|\b)", "Lodge ", x))
                      .apply(lambda x: re.sub(r"Mtn($|\b)", "Mountain ", x))
                      .apply(lambda x: re.sub(r"Mt($|\b)", "Mountain ", x))
                      .apply(lambda x: re.sub(r"Ind($|\b)", "Industrial ", x))
                      .apply(lambda x: re.sub(r"Lk($|\b)", "Lake ", x))
                      .apply(lambda x: re.sub(r"Vlg($|\b)", "Village ", x))
                      .apply(lambda x: re.sub(r"Mnr($|\b)", "Manor ", x))
                      .apply(lambda x: re.sub(r"Knls($|\b)", "Knolls ", x))
                      .apply(lambda x: re.sub(r"Knl($|\b)", "Knoll ", x))
                      .apply(lambda x: re.sub(r"Pswy($|\b)", "Passway ", x))
                      .apply(lambda x: re.sub(r"Qtr($|\b)", "Quarter ", x))
                      .apply(lambda x: re.sub(r"Mdw($|\b)", "Meadow ", x))
                      .apply(lambda x: re.sub(r"Vly($|\b)", "Valley ", x))
                      .apply(lambda x: re.sub(r"Kn($|\b)", "Knoll ", x))
                      .apply(lambda x: re.sub(r"Grn($|\b)", "Green ", x))
                      .apply(lambda x: re.sub(r"Is($|\b)", "Island ", x))
    )

In [18]:
def clean(df):
    df = df.dropna(subset="addr")
    df["housenr"] = df["addr"].apply(lambda x:np.nan if x==np.nan else re.split(r"\s|/", x)[0])
    df["housenr"] = df["housenr"].apply(match)
    df["addr_mod"] = (df["addr"].apply(split_digits)
                      .str.strip("-")
                      .apply(lambda x: x if not x.strip().endswith('La') else x.replace('La', 'Lane'))
                      .apply(lambda x: x if not x.strip().endswith('Ext') else x.replace('Ext', 'Extension'))
                      .apply(lambda x: re.sub(r"\-[A-Z]$", "",x))
                      .apply(lambda x: re.sub(r"\s[A-Z]\-[A-Z]\s", "",x))
                      .str.replace("Rt ", "Route ")
                      .str.replace('No ', 'North ')
                      .str.replace(' No ', ' North ')
                      .str.replace('Resv ', 'Reservoir ')
                      .str.replace(' Resv ', ' Reservoir ')
                      .str.replace(' Rd ', ' Road ')
                      .str.replace(' La ', ' Lane ')
                      .str.replace('Talcott Forest Road East', 'Talcott Forest Road') #???
                      .apply(lambda x: re.sub(r'\bLn(\s|$)', "Lane ", x))
                      .str.replace(' Tpke ', ' Turnpike ')
                      .apply(lambda x: re.sub(r"(^|\s)N($|\s)", " North ", x))
                      .apply(lambda x: re.sub(r"(^|\s)W($|\s)", " West ", x))
                      .apply(lambda x: re.sub(r"(^|\s)S($|\s)", " South  ", x))
                      .apply(lambda x: re.sub(r"(^|\s)E($|\s)", " East ", x))
                      .apply(lambda x: re.sub(r"(^|\s)N W($|\s)", " Northwest ", x))
                      .apply(put_the_to_front)
                      .apply(lambda x: re.sub("O Neill", "O'Neill", x))
                      .apply(lambda x: re.sub(r"O Brien", "O'Brien", x))
                      .apply(lambda x: re.sub(r"O Clock", "O'Clock", x))
                      .apply(lambda x: re.sub(r"O Rocks", "O'Rocks", x))
                      .apply(lambda x: re.sub(r"Wells Place Place", "Wells Place", x))
                      .apply(lambda x: re.sub(r"Alexander D ", "Alexander Drive ", x))
                      .apply(remove_letters)
                      .apply(lambda x: re.sub(r'\(.*?\)', "", x)) # remove ("any string")
                      .apply(remove_end_digits)
                      .apply(ad_split)
                      .apply(lambda x: re.sub(r"#\S*", "", x))
                      .str.strip()
                      .str.strip(",")
                      .str.strip(".")
                      .str.strip()
    )
    df["addr_mod"] = abbrv(df["addr_mod"])
    df["addr_mod"] = (df["addr_mod"]
                      .str.replace("#North", "North")
                      .str.replace("#South", "South")
                      .str.replace("#West", "West")
                      .str.replace("#East", "East")
                      .str.replace("#Wy", "Way")
                      .apply(remove_letters)
                      .str.replace("- Union", "Union")
                      .str.replace("- Extension", "Extension")
                      .str.replace("-Extension", "Extension")
                      .str.replace("Nrwh Wstly", "Norwich Westerly")
                      .str.replace("Avenue-Extension", "Avenue Extension")
                      .str.replace("  ", " ")
                      .str.strip("_")
                      .str.strip()
                      .apply(lambda x: re.sub(r"^[\d\s.]*", "", x))
                      .apply(lambda x: re.sub(r"Rear$", "", x))
                      .apply(lambda x: re.sub(r"Re$", "", x))
                      .apply(lambda x: re.sub(r"Rea$", "Rear", x))
                      .apply(lambda x: re.sub(r"Gar$", "", x))
                      .apply(lambda x: re.sub(r"Lt$", "", x))
                      .apply(lambda x: re.sub(r"Adj$", "", x))
                      .apply(lambda x: re.sub(r"Aka$", "", x))
                      .apply(lambda x: re.sub(r"Ch$", "", x))
                      .apply(lambda x: re.sub(r"Ch2$", "", x))
                      .apply(lambda x: re.sub(r"Ctg$", "", x))
                      .apply(lambda x: re.sub(r"Ul$", "", x))
                      .apply(lambda x: re.sub(r"Cb$", "", x))
                      .apply(lambda x: re.sub(r"Ogba$", "", x))
                      .apply(lambda x: re.sub(r"Gnh$", "", x))
                      .apply(lambda x: re.sub(r"Bpbc$", "", x))
                      .apply(lambda x: re.sub(r"Lowr$", "", x))
                      .apply(lambda x: re.sub(r"Lp$", "", x))
                      .apply(lambda x: re.sub(r"Om$", "", x))
                      .apply(lambda x: re.sub(r"Ply$", "", x))
                      .apply(lambda x: re.sub(r"Preq$", "", x))
                      .apply(lambda x: re.sub(r"Lz$", "", x))
                      .apply(lambda x: re.sub(r"Rz$", "", x))
                      .apply(lambda x: re.sub(r"Abcd$", "", x))
                      .apply(lambda x: re.sub(r"Eb$", "", x))
                      .apply(lambda x: re.sub(r"Prim$", "", x))
                      .apply(lambda x: re.sub(r"Osg$", "", x))
                      .apply(lambda x: re.sub(r"Dwl$", "", x))
                      .apply(lambda x: re.sub(r"Iii$", "", x))
                      .apply(lambda x: re.sub(r"Ab$", "", x))
                      .apply(lambda x: re.sub(r"Lot$", "", x))
                      .apply(lambda x: re.sub(r"Rr$", "", x))
                      .apply(lambda x: re.sub(r"Bldg$", "", x))
                      .apply(lambda x: re.sub(r"Beu$", "", x))
                      .apply(lambda x: re.sub(r"Na$", "", x))
                      .apply(lambda x: re.sub(r"Ph$", "", x))
                      .apply(lambda x: re.sub(r"Un1$", "", x))
                      .apply(lambda x: re.sub(r"Una$", "", x))
                      .apply(lambda x: re.sub(r"Kc$", "", x))
                      .apply(lambda x: re.sub(r"Bh$", "", x))
                      .apply(lambda x: re.sub(r"Mr$", "", x))
                      .apply(lambda x: re.sub(r"Prvt$", "", x))
                      .apply(lambda x: re.sub(r"Gnb$", "", x))
                      .apply(lambda x: re.sub(r"Bsmt$", "", x))
                      .apply(lambda x: re.sub(r"Unit$", "", x))
                      .apply(lambda x: re.sub(r"Aa$", "", x))
                      .apply(lambda x: re.sub(r"Ru$", "", x))
                      .apply(lambda x: re.sub(r"Street Land$", "Street", x)) #???
                      .apply(remove_end_digits)
                      .apply(remove_start_digits)
                      .apply(lambda x: re.sub(r"^\d+", "", x))
                      .apply(lambda x: re.sub(r"Oneill", "O'neill", x))
                      .apply(lambda x: re.sub(r"\([a-zA-Z\d]*\)", "",x)) 
                      .apply(lambda x: re.sub(r"^Off\b", " ",x)) 
                      .apply(lambda x: re.sub(r"^Rdwy", "",x)) 
                      .apply(lambda x: re.sub(r"^Lot\b", " ",x)) 
                      .apply(lambda x: re.sub(r"^Lot A", "",x)) 
                      .apply(remove_after_paran)
                      .str.replace("Fish Game", "Fish + Game")
                      .str.replace("Street Andrews", "St Andrews")
                      .str.replace("Street John", "St John")
                      .str.replace("Street Lawrence", "St Lawrence")
                      .str.replace("Street Paul", "St Paul")
                      .str.replace("Street Thomas", "St Thomas")
                      .str.replace("Street Stephens", "St Stephens")
                      .str.replace("Street Mathias", "St Stephens")
                      .str.replace("Street Andrew", "St Andrew")
                      .str.replace("Street James", "St James")
                      .str.strip()
                      .str.strip(",")
                      .str.strip("-")
                      .apply(remove_letters)
                      .str.strip("-")
                      .str.strip(",")
                      .str.strip("-")
                      .str.strip(".")
                      .str.strip("+")
                      .str.strip()
                      )
    df["housenr"] = np.where(df.housenr.str.isdigit(), df.housenr, np.nan)
    return df.reset_index(drop="True")

In [19]:
def match_adress_to_location(df, df_geo, county="all"):
    if county!="all":
        df = df[df.county == county]
    print(df.shape)
    df_merge = df.merge(df_geo, how="left", left_on=["addr_mod", "housenr", "city"], right_on=["addr", "housenr", "city"])
    df_missings = df_merge[df_merge.geometry.isnull()]
    df_found = df_merge.dropna(subset=["geometry"]) 
    df_missings = df_merge[df_merge.geometry.isnull()]
    df_missing_housnr = df_missings[df_missings["housenr"].isnull()]
    df_missings = df_missings.dropna(subset="housenr")
    df_missing_merge = df_missings.merge(df_geo, how="left", left_on=["addr_mod", "city"], right_on=["addr", "city"])

    df_missing_merge["housenr_x"] = df_missing_merge["housenr_x"].dropna().apply(lambda x: re.sub(r'[a-zA-z]', '', x)).astype("float64")
    df_missing_merge["housenr_y"] = (df_missing_merge["housenr_y"].dropna().apply(lambda x: x.split('-')[0])
                                     .apply(lambda x: x.split("+")[0])
                                     .apply(lambda x: x.split("&")[0])
                                     .apply(lambda x: x.split(";")[0])
                                     .apply(lambda x: x.split(" ")[0])
                                     .apply(lambda x: x.split(",")[0])
                                     .apply(lambda x: x.split("/")[0])
                                     .str.strip("#")
                                     .str.strip("Ë")
                                     .str.replace(")", "")
                                     .apply(lambda x: re.sub(r'[a-zA-z]', '', x)))
    df_missing_merge["housenr_y"] = np.where(df_missing_merge["housenr_y"] == "", np.nan, df_missing_merge["housenr_y"]).astype("float64")
    df_missing_merge["diff"] = np.abs(df_missing_merge["housenr_x"] - df_missing_merge["housenr_y"])
    df_found_2 = df_missing_merge.loc[df_missing_merge.dropna(subset="geometry_y").groupby(by=["addr_x", "city"])["diff"].idxmin().dropna()]
    df_missing_merge_h = df_missing_housnr.merge(df_geo, how="left", left_on=["addr_mod", "city"], right_on=["addr", "city"])
    df_missing_merge_h = df_missing_merge_h.dropna(subset="geometry_y")
    df_found_3 = df_missing_merge_h.dropna(subset="geometry_y").groupby(by=["addr_x", "city"]).sample(1)
    selected_columns = ["addr", "city", "county", "state", "geometry"]

    
    df_found = df_found.rename(columns={"geometry_y": "geometry", "addr_x":"addr"})
    df_found_2 = df_found_2.drop(columns="addr").rename(columns={"geometry_y": "geometry", "addr_x":"addr"})
    df_found_3 = df_found_3.drop(columns="addr").rename(columns={"geometry_y": "geometry", "addr_x":"addr"})

    df_clean = pd.concat([df_found, df_found_2, df_found_3])
    return df_clean[selected_columns].reset_index(drop=True).drop_duplicates(subset=["addr", "city"])

#### Clean Data

In [20]:
df_combined = load_data_to_geocode()
df_cleaned = clean(df_combined)

In [21]:
# clean
df_cleaned = clean(df_combined)

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_45925/2622593439.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["housenr"] = df["addr"].apply(lambda x:np.nan if x==np.nan else re.split(r"\s|/", x)[0])
/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_45925/2622593439.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["housenr"] = df["housenr"].apply(match)
/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_45925/2622593439.py:5: SettingWithCopyWarning: 
A value is try

#### Match

##### Fairfax

In [22]:
df_fairfax = df_cleaned[df_cleaned.county == "Fairfax"]

In [24]:
df_geo_fairfax = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_fairfax1_2.csv", index_col=[0])

In [27]:
# geo data prep use highway data
df_fairfax_cities = gpd.read_file("../data/raw/geocoding/city_boundries/fairfax_cities.geojson", index_col=[0])
df_highway_fairfax = pd.read_csv("/Volumes/Seagate/Bavillion/geo/highway/fairfax_highway.csv")
df_highway_fairfax = df_highway_fairfax[["geometry", "tiger:county", "name"]].dropna(subset="name")
df_highway_fairfax = df_highway_fairfax.dropna(subset="geometry")
df_highway_fairfax["geometry"] = df_highway_fairfax["geometry"].apply(wkt.loads)
df_highway_fairfax = gpd.GeoDataFrame(df_highway_fairfax)
df_highway_fairfax.crs = "EPSG:4326"
df_highway_fairfax = df_highway_fairfax.sjoin(df_fairfax_cities[["ZIPCODE", "ZIPCITY", "geometry"]], how="left", predicate="intersects")
df_highway_fairfax = df_highway_fairfax.drop(columns=["tiger:county", "index_right"]).rename(columns={"ZIPCODE":"addr:postcode", 
                                "ZIPCITY":"city", 
                                "name": "addr"})
df_highway_fairfax["city"] = df_highway_fairfax["city"].str.title()
df_highway_fairfax["housenr"] = np.nan

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_45925/2261432812.py:2: DtypeWarning: Columns (0,3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,37,38,39,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,130,133,134,135,137,138,139,140,142,143,144,145,146,147,148,149,151,152,153,154,155,156,157,158,159,160,161,162,167,168,169,171,172,174,175,176,178,179,180,181,182,183,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,206,207,208,209,210,211,212,213,214,215,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,234,235,236,237,238,239,240,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,275,276,277,27

,geometry,addr,addr:postcode,city,housenr
1,"LINESTRING (-77.42476 38.91490, -77.42492 38.9...",Air and Space Museum Parkway,20151.0,Chantilly,NaN
3,"LINESTRING (-77.26545 38.86497, -77.26520 38.8...",Arlington Boulevard,22031.0,Fairfax,NaN
4,"LINESTRING (-77.30880 38.84284, -77.30888 38.8...",Judicial Drive,22030.0,Fairfax,NaN
5,"LINESTRING (-77.32029 38.83755, -77.31208 38.8...",University Drive,22030.0,Fairfax,NaN
7,"LINESTRING (-77.29778 38.77583, -77.29402 38.7...",Lakehaven Court,22015.0,Burke,NaN
...,...,...,...,...,...
163052,POINT (-77.36321 38.95386),SUNSET HILLS RD @ TOWN CENTER PKWY,20190.0,Reston,NaN
163053,POINT (-77.36478 38.95417),SUNSET HILLS RD @ TOWN CTR PKW,20190.0,Reston,NaN
163170,POINT (-77.14519 38.75675),Kingstowne Village Pkwy. and Hayfield Rd.,22315.0,Alexandria,NaN
163187,POINT (-77.15143 38.75349),Kingstowne Village Pkwy. and Park Village Dr.,22315.0,Alexandria,NaN


In [28]:
# preparing geo data
df_geo_fairfax["geometry"] = df_geo_fairfax["geometry"].apply(wkt.loads)
df_geo_fairfax = gpd.GeoDataFrame(df_geo_fairfax)
df_geo_fairfax.crs = "EPSG:4326" 

df_fairfax_cities.crs = "EPSG:4326" 

df_geo_fairfax = df_geo_fairfax.sjoin(df_fairfax_cities[["ZIPCODE", "ZIPCITY", "geometry"]], how="left", predicate="intersects")
df_geo_fairfax = df_geo_fairfax.drop(columns=["city", "addr:postcode"]).rename(columns={"ZIPCODE": "postcode", "ZIPCITY":"city"})

df_geo_fairfax["city"] = df_geo_fairfax["city"].str.title()

grouped_addr = df_fairfax.groupby(by=["addr_mod", "city"]).size().to_frame()
grouped_addr.columns = ["size"]
grouped_addr = grouped_addr.reset_index()
addr_count = grouped_addr.addr_mod.value_counts().to_frame().reset_index()
unique_addr = addr_count[addr_count["count"] == 1].addr_mod.values
unique_addr_city = df_fairfax[df_fairfax.addr_mod.isin(unique_addr)][["addr_mod", "city"]].drop_duplicates()
df_geo_fairfax = df_geo_fairfax.merge(unique_addr_city, how="left", left_on="addr", right_on="addr_mod", suffixes=["_geo", "_add"])
df_geo_fairfax["city"] = np.where(df_geo_fairfax.city_geo.isna(), df_geo_fairfax.city_add, df_geo_fairfax.city_geo)
df_geo_fairfax = df_geo_fairfax.drop(columns=["city_geo", "city_add", "addr_mod"])

In [29]:
def _split(postcode):
    if isinstance(postcode, float) and np.isnan(postcode):
        return postcode
    return postcode.split("-")[0]


In [30]:
# find how many are matched on whole address data
df_merge = df_fairfax.merge(df_geo_fairfax, how="left", left_on=["addr_mod", "housenr", "city"], right_on=["addr", "housenr", "city"])

In [35]:
df_final_match = pd.read_csv("../data/cleaned/final_match.csv")

##### Windham

In [41]:
connecticut_zip = gpd.read_file("../data/raw/geocoding/city_boundries/ct_connecticut_zip_codes_geo.min.json")
connecticut_city = pd.read_csv("../data/raw/geocoding/city_boundries/ct_zipcode_city.csv")
connecticut_zip = connecticut_zip[["ZCTA5CE10", "geometry"]]
connecticut_city = connecticut_city[["zip", "City"]]
connecticut_city = connecticut_city.dropna(subset="City")
connecticut_zip = connecticut_zip.rename(columns={"ZCTA5CE10":"zip"})
connecticut_zip["zip"] = connecticut_zip.zip.str.lstrip("0")
connecticut_zip = connecticut_zip.merge(connecticut_city, how="left", on="zip")
connecticut_zip["City"] = connecticut_zip.City.str.title()

In [42]:
df_windham = df_cleaned[df_cleaned.county == "Windham"]
df_geo_windham = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_windham_2.csv", index_col=[0])

In [43]:
# find how many are matched on whole address data
df_merge = df_windham.merge(df_geo_windham, how="left", left_on=["addr_mod", "housenr", "city"], right_on=["addr", "housenr", "city"])
print(df_merge.shape)
print(df_merge.dropna(subset="geometry").shape)
df_final_match = pd.read_csv("../data/cleaned/final_match.csv")
print(df_final_match[df_final_match.county == "Windham"].shape)

(2888, 9)
(2739, 9)
(2704, 6)


In [59]:
# find how many are matched on whole address data
def get_match_numbers(county, df, df_geo):
    df_merge = df.merge(df_geo, how="left", left_on=["addr_mod", "housenr", "city"], right_on=["addr", "housenr", "city"])
    df_merge = df_merge.drop_duplicates(subset=["addr_x", "county", "housenr"])
    print(df.shape)
    print(df_merge.dropna(subset="geometry").shape)
    df_final_match = pd.read_csv("../data/cleaned/final_match.csv")
    print(df_final_match[df_final_match.county == county].shape[0] - df_merge.dropna(subset="geometry").shape[0])
    print(df_final_match[df_final_match.county == county].shape[0] - df_merge.shape[0])

##### Tolland

In [44]:
df_tolland = df_cleaned[df_cleaned.county == "Tolland"]
df_geo_tolland = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_tolland_2.csv", index_col=[0])

In [46]:
# find how many are matched on whole address data
df_merge = df_tolland.merge(df_geo_tolland, how="left", left_on=["addr_mod", "housenr", "city"], right_on=["addr", "housenr", "city"])
print(df_merge.shape)
print(df_merge.dropna(subset="geometry").shape)
df_final_match = pd.read_csv("../data/cleaned/final_match.csv")
print(df_final_match[df_final_match.county == "Tolland"].shape[0] - df_merge.dropna(subset="geometry").shape[0])
print(df_final_match[df_final_match.county == "Tolland"].shape[0] - df_merge.shape[0])

(22187, 9)
(13651, 9)


6383
-2153


In [58]:
# find how many are matched on whole address data
df_merge = df_tolland.merge(df_geo_tolland, how="left", left_on=["addr_mod", "housenr", "city"], right_on=["addr", "housenr", "city"])
df_merge = df_merge.drop_duplicates(subset=["addr_x", "county", "housenr"])
print(df_tolland.shape)
print(df_merge.dropna(subset="geometry").shape)
df_final_match = pd.read_csv("../data/cleaned/final_match.csv")
print(df_final_match[df_final_match.county == "Tolland"].shape[0] - df_merge.dropna(subset="geometry").shape[0])
print(df_final_match[df_final_match.county == "Tolland"].shape[0] - df_merge.shape[0])

(22162, 6)
(13561, 9)
6473
-2049


##### New London

In [47]:
df_new_london = df_cleaned[df_cleaned.county == "New London"]
df_geo_london = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_new_london_2.csv", index_col=[0])

##### New Haven

In [62]:
df_new_haven = df_cleaned[df_cleaned.county == "New Haven"]
df_geo_haven = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_new_haven_2.csv", index_col=[0])

In [63]:
get_match_numbers("New Haven", df_new_haven, df_geo_haven)

(154534, 6)
(125557, 9)
26448
861


##### Litchfield

In [64]:
df_litchfield = df_cleaned[df_cleaned.county == "Litchfield"]
df_geo_litchfield = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_litchfield_2.csv", index_col=[0])

In [65]:
get_match_numbers("Litchfield", df_litchfield, df_geo_litchfield)

(36260, 6)
(31700, 9)
3822
-369


##### Hartford

In [66]:
df_hartford = df_cleaned[df_cleaned.county == "Hartford"]
df_geo_hartford = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_hartford_2.csv", index_col=[0])

/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_45925/1215722233.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo_hartford = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_hartford_2.csv", index_col=[0])


In [67]:
get_match_numbers("Hartford", df_hartford, df_geo_hartford)

(157825, 6)
(136549, 9)
17704
-1344


##### Middlesex

In [68]:
df_middlesex = df_cleaned[df_cleaned.county == "Middlesex"]
df_geo_middlesex = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_middlesex_2.csv", index_col=[0])

In [69]:
get_match_numbers("Middlesex", df_middlesex, df_geo_middlesex)

(5246, 6)
(4697, 9)
461
-88


##### Fairfield

In [70]:
df_fairfield = df_cleaned[df_cleaned.county == "Fairfield"]
df_geo_fairfield = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_fairfield_2.csv", index_col=[0])


/var/folders/ch/yj3xbk4s1t34gtb89jtxcmcr0000gn/T/ipykernel_45925/949862202.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_geo_fairfield = pd.read_csv("../data/raw/geocoding/houses_geocoordinates/df_geo_fairfield_2.csv", index_col=[0])


In [71]:
get_match_numbers("Fairfield", df_fairfield, df_geo_fairfield)

(130884, 6)
(113600, 9)
16028
-161


#### Merge together

In [ ]:
final_match = pd.concat([
    final_match_fairfield,
    final_match_fairfax,
    final_match_hartford,
    final_match_haven,
    final_match_litchfield,
    final_match_london,
    final_match_middlesex,
    final_match_tolland,
    final_match_windham,
])

In [ ]:
final_match.reset_index(drop=True, inplace=True)

In [ ]:
final_match.to_csv("final_match.csv")

##### Missings

In [ ]:
missing = pd.DataFrame()
counties = {
            "new_haven" : "New Haven",
            "new_london": "New London",
            "middlesex": "Middlesex",
            "litchfield": "Litchfield",
            "hartford": "Hartford",
            "fairfield": "Fairfield",
            "tolland": "Tolland",
            "windham": "Windham",
            }

for k, v in counties.items():
    df = df_cleaned[df_cleaned.county == v]
    df_geo = pd.read_csv(f"df_geo_{k}_2.csv", index_col=[0])
    match = match_adress_to_location(df_cleaned, df_geo, county=v)
    missing = pd.concat([missing, df[~df.addr.isin(match.addr)][["addr_mod", "addr", "city", "county"]].drop_duplicates(subset="addr_mod")])

In [ ]:
a = pd.read_csv("df_geo_new_haven_2.csv", index_col=[0])
b = pd.read_csv("df_geo_new_london_2.csv", index_col=[0])
c = pd.read_csv("df_geo_middlesex_2.csv", index_col=[0])
d = pd.read_csv("df_geo_litchfield_2.csv", index_col=[0])
e = pd.read_csv("df_geo_hartford_2.csv", index_col=[0])
f = pd.read_csv("df_geo_fairfield_2.csv", index_col=[0])
g = pd.read_csv("df_geo_tolland_2.csv", index_col=[0])
h = pd.read_csv("df_geo_windham_2.csv", index_col=[0])

df = df_cleaned[df_cleaned.county != "Fairfax"]
df_geo = pd.concat([a, b,c,d,e,f,g,h])
match = match_adress_to_location(df_cleaned, df_geo)
missing = df[~df.addr.isin(match.addr)][["addr_mod", "addr", "city", "county"]].drop_duplicates(subset="addr_mod")

In [ ]:
missing.shape